<a href="https://colab.research.google.com/github/Navu45/object-detection/blob/master/ssd_mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Mobilenet v2 Quantization with ONNX Runtime on CPU

In this tutorial, we will load a mobilenet v2 model pretrained with [PyTorch](https://pytorch.org/), export the model to ONNX, and quantize then run with ONNXRuntime.

## 0. Prerequisites ##

If you have Jupyter Notebook, you can run this notebook directly with it. You may need to install or upgrade [PyTorch](https://pytorch.org/), [OnnxRuntime](https://microsoft.github.io/onnxruntime/), and other required packages.

Otherwise, you can setup a new environment. First, install [Anaconda](https://www.anaconda.com/distribution/). Then open an AnaConda prompt window and run the following commands:

```console
conda create -n cpu_env python=3.8
conda activate cpu_env
conda install jupyter
jupyter notebook
```
The last command will launch Jupyter Notebook and we can open this notebook in browser to continue.

### 0.1 Install packages
Let's install the necessary packages to start the tutorial. We will install PyTorch 1.8, OnnxRuntime 1.8, latest ONNX and pillow.

In [2]:
# Install or upgrade PyTorch 1.8.0 and OnnxRuntime 1.8 for CPU-only.
import sys
!{sys.executable} -m pip install --upgrade torch==1.8.0 torchvision==0.9.0 torchaudio===0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
!{sys.executable} -m pip install --upgrade onnxruntime==1.8.0
!{sys.executable} -m pip install --upgrade onnx
!{sys.executable} -m pip install --upgrade pillow

Looking in links: https://download.pytorch.org/whl/torch_stable.html


ERROR: Could not find a version that satisfies the requirement torch==1.8.0 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu113, 1.11.0+cu115, 1.12.0, 1.12.0+cpu, 1.12.0+cu113, 1.12.0+cu116, 1.12.1, 1.12.1+cpu, 1.12.1+cu113, 1.12.1+cu116, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117)
ERROR: No matching distribution found for torch==1.8.0
ERROR: Could not find a version that satisfies the requirement onnxruntime==1.8.0 (from versions: 1.12.0, 1.12.1, 1.13.1)
ERROR: No matching distribution found for onnxruntime==1.8.0


# 1 Download pretrained model and export to ONNX

In this step, we load a pretrained mobilenet v2 model, and export it to ONNX.

### 1.1 Load the pretrained model
Use torchvision provides API to load mobilenet_v2 model.

In [7]:
from torchvision import models, datasets, transforms as T
ssdlite_mobilenet_v3 = models.detection.ssdlite320_mobilenet_v3_large(pretrained=True)

C:\Users\Alexey\anaconda3\envs\deep-learning-mptu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Alexey\anaconda3\envs\deep-learning-mptu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSDLite320_MobileNet_V3_Large_Weights.COCO_V1`. You can also use `weights=SSDLite320_MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### 1.2 Export the model to ONNX
Pytorch onnx export API to export the model.

In [13]:
import torch
image_height = 320
image_width = 320
x = torch.randn(1, 3, image_height, image_width, requires_grad=True)
ssdlite_mobilenet_v3.eval()
with torch.no_grad():
  torch_out = ssdlite_mobilenet_v3(x)
  # Export the model
  torch.onnx.export(ssdlite_mobilenet_v3,              # model being run
                    x,                         # model input (or a tuple for multiple inputs)
                    "ssdlite_mobilenet_v3.onnx", # where to save the model (can be a file or file-like object)
                    export_params=True,        # store the trained parameter weights inside the model file
                    opset_version=12,          # the ONNX version to export the model to
                    do_constant_folding=True,  # whether to execute constant folding for optimization
                    input_names = ['input'],   # the model's input names
                    output_names = ['output']) # the model's output names


### 1.3 Sample Execution with ONNXRuntime

Run an sample with the full precision ONNX model. Firstly, implement the preprocess.

In [15]:
from PIL import Image
import numpy as np
import onnxruntime
import torch

def preprocess_image(image_path, height, width, channels=3):
    image = Image.open(image_path)
    image = image.resize((width, height), Image.ANTIALIAS)
    image_data = np.asarray(image).astype(np.float32)
    image_data = image_data.transpose([2, 0, 1]) # transpose to CHW
    mean = np.array([0.079, 0.05, 0]) + 0.406
    std = np.array([0.005, 0, 0.001]) + 0.224
    for channel in range(image_data.shape[0]):
        image_data[channel, :, :] = (image_data[channel, :, :] / 255 - mean[channel]) / std[channel]
    image_data = np.expand_dims(image_data, 0)
    return image_data

#### Download the imagenet labels and load it

In [16]:
# Download ImageNet labels
!curl -o imagenet_classes.txt https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 10472  100 10472    0     0  35339      0 --:--:-- --:--:-- --:--:-- 35378


#### Run the example with ONNXRuntime

In [19]:
session_fp32 = onnxruntime.InferenceSession("ssdlite_mobilenet_v3.onnx")

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def run_sample(session, image_file, categories):
    output = session.run([], {'input':preprocess_image(image_file, image_height, image_width)})[0]
    output = output.flatten()
    output = softmax(output) # this is optional
    top5_catid = np.argsort(-output)[:5]
    for catid in top5_catid:
        print(categories[catid], output[catid])

run_sample(session_fp32, 'cat.jpg', categories)

banded gecko 0.03943445
jellyfish 0.03943445
wombat 0.03943445
oxcart 0.03943445
leatherback turtle 0.03943445


C:\Users\Alexey\AppData\Local\Temp\ipykernel_7900\2249352223.py:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize((width, height), Image.ANTIALIAS)


# 2 Quantize the model with ONNXRuntime 
In this step, we load the full precison model, and quantize it with ONNXRuntime quantization tool. And show the model size comparison between full precision and quantized model. Finally, we run the same sample with the quantized model

## 2.1 Implement a CalibrationDataReader
CalibrationDataReader takes in calibration data and generates input for the model

In [20]:
from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantType
import os

def preprocess_func(images_folder, height, width, size_limit=0):
    image_names = os.listdir(images_folder)
    if size_limit > 0 and len(image_names) >= size_limit:
        batch_filenames = [image_names[i] for i in range(size_limit)]
    else:
        batch_filenames = image_names
    unconcatenated_batch_data = []

    for image_name in batch_filenames:
        image_filepath = images_folder + '/' + image_name
        image_data = preprocess_image(image_filepath, height, width)
        unconcatenated_batch_data.append(image_data)
    batch_data = np.concatenate(np.expand_dims(unconcatenated_batch_data, axis=0), axis=0)
    return batch_data


class MobilenetDataReader(CalibrationDataReader):
    def __init__(self, calibration_image_folder):
        self.image_folder = calibration_image_folder
        self.preprocess_flag = True
        self.enum_data_dicts = []
        self.datasize = 0

    def get_next(self):
        if self.preprocess_flag:
            self.preprocess_flag = False
            nhwc_data_list = preprocess_func(self.image_folder, image_height, image_width, size_limit=0)
            self.datasize = len(nhwc_data_list)
            self.enum_data_dicts = iter([{'input': nhwc_data} for nhwc_data in nhwc_data_list])
        return next(self.enum_data_dicts, None)

## 2.2 Quantize the model

As we can not upload full calibration data set for copy right issue, we only demonstrate with some example images. You need to use your own calibration data set in practice.

In [21]:
!python -m wget https://github.com/microsoft/onnxruntime-inference-examples/tree/main/quantization/notebooks/imagenet_v2/calibration_imagenet

Traceback (most recent call last):
  File "C:\Users\Alexey\anaconda3\envs\deep-learning-mptu\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Alexey\anaconda3\envs\deep-learning-mptu\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\Alexey\anaconda3\envs\deep-learning-mptu\lib\site-packages\wget.py", line 568, in <module>
    filename = download(args[0], out=options.output)
  File "C:\Users\Alexey\anaconda3\envs\deep-learning-mptu\lib\site-packages\wget.py", line 533, in download
    filename = filename_fix_existing(filename)
  File "C:\Users\Alexey\anaconda3\envs\deep-learning-mptu\lib\site-packages\wget.py", line 269, in filename_fix_existing
    name, ext = filename.rsplit('.', 1)
ValueError: not enough values to unpack (expected 2, got 1)


In [ ]:
# change it to your real calibration data set
calibration_data_folder = "calibration_imagenet"
dr = MobilenetDataReader(calibration_data_folder)

quantize_static('mobilenet_v2_float.onnx',
                'mobilenet_v2_uint8.onnx',
                dr)

print('ONNX full precision model size (MB):', os.path.getsize("mobilenet_v2_float.onnx")/(1024*1024))
print('ONNX quantized model size (MB):', os.path.getsize("mobilenet_v2_uint8.onnx")/(1024*1024))

C:\Users\Alexey\AppData\Local\Temp\ipykernel_4724\2249352223.py:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize((width, height), Image.ANTIALIAS)


ONNX full precision model size (MB): 13.344060897827148
ONNX quantized model size (MB): 3.4843339920043945


## 2.3 Run the model with OnnxRuntime

In [ ]:
session_quant = onnxruntime.InferenceSession("mobilenet_v2_uint8.onnx")
run_sample(session_quant, 'cat.jpg', categories)

tabby 0.71912986
tiger cat 0.1969283
Egyptian cat 0.07271367
lynx 0.0033133852
tiger 0.002999183


C:\Users\Alexey\AppData\Local\Temp\ipykernel_4724\2249352223.py:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize((width, height), Image.ANTIALIAS)
